In [120]:
import vk_api
import io
import json, codecs

In [128]:
# credentials: нужно указать логин/пароль к vk.com
# чтобы случайно не закомитить версию с логин/пароль - выносим эти данные в отдельный файл, который исключаем в .gitignore
credentialsFileName = "vk_login_password.txt" # формат файла: 1-я строка = логин, 2-я строка = пароль
# затираем значения, которые могли сохраниться с прошлых запусков (в notebook)
login = '+71231234567' 
password = ''

# загружаем значения из файла
try:
    loginFile = open(credentialsFileName, 'r')
    login = loginFile.readline().strip() # в формате: '+71231234567'
    password = loginFile.readline().strip()
except IOError:
    print ("No file with login/password: создайте файл " + credentialsFileName + ", в котором 1-я строка = логин, 2-я строка = пароль")

In [ ]:
# используем:
#  wrapper-lib vk_api: https://github.com/python273/vk_api
#  описание Api от vk: https://vk.com/dev/groups.search?params[q]=bitcoin&params[future]=0&params[market]=0&params[offset]=3&params[count]=3&params[v]=5.92

In [50]:
# получаем соединение, api
vk_session = vk_api.VkApi(login, password)
vk_session.auth(token_only=True)
api = vk_session.get_api()

In [130]:
# Данные по сообществам - получаем по http
def obtainGroups(api, keyword, count):
    print("obtainGroups, keyword=" + keyword + ", count=" + str(count))
    results = api.groups.search(q=keyword, count=count)
    return results['items']

In [129]:
# получить N записей со стены сообщества/пользователя
def obtainWallItems(api, owner_id, count):
    # protection: проверяем входные значения
    if(count > 100):
        raise AssertionError("count должен быть <= 100, но передали:" + str(count))

    # основная часть
    results = api.wall.get(owner_id=owner_id, filter = 'owner', count = count, offset=0)
    print("obtainWallItems, owner_id=" + str(owner_id) + "retrieve " + str(len(results['items'])) + ", of total " + str(results['count']))
    return results['items']

In [72]:
def writeJsonToFile(fileName, jsonData):
    print(json.dumps(jsonData))
    with open(fileName, 'w') as outfile:
        json.dump(jsonData, outfile, ensure_ascii=False, sort_keys=True, indent=4)

In [77]:
# записываем json в файл
def writeJsonToFile2(fileName, jsonData):
    print(json.dumps(jsonData))
    with io.open('data.txt', 'w', encoding='utf-8') as f:
        f.write(json.dumps(jsonData, ensure_ascii=False))

In [131]:
groups = obtainGroups(api, 'Bitcoin', 100)
print(groups)
print(type(groups))
writeJsonToFile2('some_file.txt', groups)

obtainGroups, keyword=Bitcoin, count=2
[{'id': 46371384, 'name': 'АБ | Криптовалюта, блокчейн, Биткоин Bitcoin BTC', 'screen_name': 'business.place', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-6.userapi.com/Yjaywg-VZOWKoid2XJuzg6Ue4h933zOW8RJFZA/hRUN7N78-00.jpg?ava=1', 'photo_100': 'https://sun1-11.userapi.com/ZtDZPaxDYxXrfDIg5kEqLCOn5D1jnlGS9TTMEA/8YUEqM2dEVk.jpg?ava=1', 'photo_200': 'https://sun1-12.userapi.com/sTPmZ80mPz2TqbrqpQD5e1SqvqL3fBXnmIku8Q/1lXPP1acyBQ.jpg?ava=1'}, {'id': 92718200, 'name': 'Brain: криптовалюта финансы биткоин Bitcoin BTC', 'screen_name': 'smart_brain7', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-1.userapi.com/c627920/v627920444/b751/AzyxT3syxrY.jpg?ava=1', 'photo_100': 'https://sun1-20.userapi.com/c627920/v627920444/b750/3gCKVnF3YfI.jpg?ava=1', 'photo_200': 'https://sun1-6.userapi.com/c627920/v627920444/b74f/z3wtAmEDthQ.jpg?ava=

In [55]:
groups = obtainGroups(api=api, keyword='bitcoin', count=2)
print(groups)

obtainGroups, keyword=bitcoin
{'count': 2869, 'items': [{'id': 46371384, 'name': 'АБ | Криптовалюта, блокчейн, Биткоин Bitcoin BTC', 'screen_name': 'business.place', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-6.userapi.com/Yjaywg-VZOWKoid2XJuzg6Ue4h933zOW8RJFZA/hRUN7N78-00.jpg?ava=1', 'photo_100': 'https://sun1-11.userapi.com/ZtDZPaxDYxXrfDIg5kEqLCOn5D1jnlGS9TTMEA/8YUEqM2dEVk.jpg?ava=1', 'photo_200': 'https://sun1-12.userapi.com/sTPmZ80mPz2TqbrqpQD5e1SqvqL3fBXnmIku8Q/1lXPP1acyBQ.jpg?ava=1'}, {'id': 92718200, 'name': 'Brain: криптовалюта финансы биткоин Bitcoin BTC', 'screen_name': 'smart_brain7', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-1.userapi.com/c627920/v627920444/b751/AzyxT3syxrY.jpg?ava=1', 'photo_100': 'https://sun1-20.userapi.com/c627920/v627920444/b750/3gCKVnF3YfI.jpg?ava=1', 'photo_200': 'https://sun1-6.userapi.com/c627920/v627920444/b74f/z3wt

In [41]:
obtainWallItems(api=vk_session.get_api(), count=1, owner_id=-92718200)

obtainWallItems, owner_id=-92718200
retrieve 1, of total 10553


[{'id': 54244,
  'from_id': -92718200,
  'owner_id': -92718200,
  'date': 1549807801,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Экспресс доставка в действии:)😂',
  'attachments': [{'type': 'doc',
    'doc': {'id': 490412771,
     'owner_id': 374134445,
     'title': 'Большой прикол',
     'size': 6596066,
     'ext': 'gif',
     'url': 'https://vk.com/doc374134445_490412771?hash=73e3fdd6ec5b388e7f&dl=GY2TCNZUGMYTI:1549811031:029939ff244955263f&api=1&no_preview=1',
     'date': 1549659202,
     'type': 3,
     'preview': {'photo': {'sizes': [{'src': 'https://pp.userapi.com/c848428/u114165366/d5/-3/m_e4d31e93bf.jpg',
         'width': 130,
         'height': 100,
         'type': 'm'},
        {'src': 'https://pp.userapi.com/c848428/u114165366/d5/-3/s_e4d31e93bf.jpg',
         'width': 100,
         'height': 75,
         'type': 's'},
        {'src': 'https://pp.userapi.com/c848428/u114165366/d5/-3/o_e4d31e93bf.jpg',
         'width': 252,
         'height': 421,
         

In [2]:
#vk_session = vk_api.VkApi('+79055057083', 'Der1PaRol')
#vk_session.auth()
#vk = vk_session.get_api()

In [14]:
vk_session = vk_api.VkApi(login, password)
vk_session.auth(token_only=True)
tools = vk_api.VkTools(vk_session)
wall = tools.get_all('wall.get', 10, {'owner_id': -92718200, 'filter':'owner'})
print('Posts count:', wall['count'])

if wall['count']:
    print('First post:', wall['items'][0], '\n')
    
if wall['count'] > 1:
    print('Last post:', wall['items'][-1])

Posts count: 10551
First post: {'id': 54241, 'from_id': -92718200, 'owner_id': -92718200, 'date': 1549800600, 'marked_as_ads': 0, 'post_type': 'post', 'text': 'Ну как готовы?', 'attachments': [{'type': 'link', 'link': {'url': 'https://m.vk.com/@-92718200-analitik-fundstrat-predskazyvaet-testirovanie-kriptorynkom-n', 'title': 'Аналитик Fundstrat предсказывает тестирование крипторынком новых минимумов', 'caption': 'm.vk.com', 'description': 'Статья', 'photo': {'id': 456244568, 'album_id': -66, 'owner_id': -92718200, 'user_id': 100, 'sizes': [{'type': 's', 'url': 'https://pp.userapi.com/c850136/v850136251/d5539/ERwoQFd8wnA.jpg', 'width': 75, 'height': 39}, {'type': 'm', 'url': 'https://pp.userapi.com/c850136/v850136251/d553a/KNTNVLDGGO0.jpg', 'width': 130, 'height': 68}, {'type': 'x', 'url': 'https://pp.userapi.com/c850136/v850136251/d553b/q1OSAEejiNM.jpg', 'width': 604, 'height': 318}, {'type': 'y', 'url': 'https://pp.userapi.com/c850136/v850136251/d553c/dbQymREhCkU.jpg', 'width': 640, '

In [31]:
vk_session = vk_api.VkApi(login, password)
vk_session.auth(token_only=True)
#tools = vk_api.VkTools(vk_session)
api = vk_session.get_api()
wall = api.wall.get(owner_id=-92718200, filter = 'owner', count = 100, offset=0)
#wall = tools.get_all('wall.get', 10, {'owner_id': -92718200, 'filter':'owner'})
print('Posts count:', wall['count'])
print('result.size:', len(wall['items']))

if wall['count']:
    print('First post:', wall['items'][0], '\n')
    
if wall['count'] > 1:
    print('Last post:', wall['items'][-1])

Posts count: 10551
result.size: 100
First post: {'id': 54241, 'from_id': -92718200, 'owner_id': -92718200, 'date': 1549800600, 'marked_as_ads': 0, 'post_type': 'post', 'text': 'Ну как готовы?', 'attachments': [{'type': 'link', 'link': {'url': 'https://m.vk.com/@-92718200-analitik-fundstrat-predskazyvaet-testirovanie-kriptorynkom-n', 'title': 'Аналитик Fundstrat предсказывает тестирование крипторынком новых минимумов', 'caption': 'm.vk.com', 'description': 'Статья', 'photo': {'id': 456244568, 'album_id': -66, 'owner_id': -92718200, 'user_id': 100, 'sizes': [{'type': 's', 'url': 'https://pp.userapi.com/c850136/v850136251/d5539/ERwoQFd8wnA.jpg', 'width': 75, 'height': 39}, {'type': 'm', 'url': 'https://pp.userapi.com/c850136/v850136251/d553a/KNTNVLDGGO0.jpg', 'width': 130, 'height': 68}, {'type': 'x', 'url': 'https://pp.userapi.com/c850136/v850136251/d553b/q1OSAEejiNM.jpg', 'width': 604, 'height': 318}, {'type': 'y', 'url': 'https://pp.userapi.com/c850136/v850136251/d553c/dbQymREhCkU.jpg'

In [3]:
print(vk)

In [ ]:
print(vk.wall.post(message='Hello world!'))